# Team Members

- Harsh Mehta _(hdm564)_
- Dipali Pandey _(dp33957)_
- Rohitashwa Chakraborty _(rc47878)_


Things to work on:
- daily data -> monthly data
- $\beta_D$ is 0
- $r_f$ is 0
- d/e ratio assumed const

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown as md, Latex, Math

import statsmodels.api as sm
from patsy import dmatrices
import yfinance as yf
from datetime import datetime

import wrds
connection = wrds.Connection()

%matplotlib inline
plt.style.use("bmh")

Loading library list...
Done


# Inputs

## Hard-coded Inputs

In [27]:
stocks = "MSFT".split()
start = pd.to_datetime("2015-01-01").date()
# end = pd.to_datetime("2020-01-01")
end = pd.to_datetime(datetime.now()).date()
period = "1mo"

## User Inputs

In [28]:
# stocks = str(input("Space separated Stock Ticker(s):")).upper().split()
# market = str(input("market Ticker: ")).upper()
# tbonds = str(input("tbonds Ticker: ")).upper()

# start = pd.to_datetime(input("Start Date: "))
# end = pd.to_datetime(input("End Date: "))

# period = str(input("Frequency:"))

## Selected Inputs

In [29]:
print("Selected stock tickers:", stocks)
# print("market Ticker: ",market)
# print("tbonds Ticker: ",tbonds)
print("Start Date: ",start)
print("End Date: ",end)
print("Frequency:", period)

Selected stock tickers: ['MSFT']
Start Date:  2015-01-01
End Date:  2021-11-08
Frequency: 1mo


# Security Class

In [33]:
class Security():

    @classmethod
    def __init__(self, *args, **kwargs)-> None:
        if "market" in kwargs.keys():
            self._market = kwargs["market"]
            try:
                self._market_permno = self.__get_permno(self, kwargs["market"])
            except:
                raise Exception("Invalid Initialisation. Need A Market Proxy")
        else:
            self._market = "SPY"
            self._market_permno = 84398
        
        if "rf" in kwargs.keys():
            self._rf = kwargs["rf"]
            try:
                self._rf_permno = self.__get_permno(self, kwargs["rf"])
            except:
                raise Exception("Invalid Initialisation. Need A Risk Free Proxy")
        else:
            self._rf = "IRX"
            self._rf_permno = 1
        
        if "start_date" in kwargs.keys():
            self._start_date = kwargs['start_date']
        else:
            self._start_date = (pd.to_datetime(datetime.now()) - pd.DateOffset(years = 5)).date()
        
        if "end_date" in kwargs.keys():
            self._end_date = kwargs['end_date']
        else:
            self._end_date = pd.to_datetime(datetime.now()).date()
        
        if isinstance(args[0], str):
            self.ticker = args[0]
            try:
                display(md("......Fetching Company Info......"))
                self.__get_company_info(self)
            except:
                display(md("–––––––––Failed to Fetch Ticker Information–––––––––"))
                print(sys.exc_info()[1])
            
            try:
                display(md("......Fetching Comps Info......"))
                self.__get_comps_info(self)
            except:
                display(md("–––––––––Failed to Fetch Comps Information–––––––––"))
                print(sys.exc_info()[1])
        # elif isinstance(args[0], pd.Series):
        #     print(args[0].columns)
        #     self.permno, self.gvkey, self.ticker, self.name, self.public_date, self.mrktcap, self.efftax,\
        #         self.de_ratio, self.sic, self.spcsrc, self.ggroup, self.gind, self.gsector, self.gsubind = args[0]
        
        try:
            display(md("......Calculating Beta......"))
            self.beta = self.__getbeta(self, self.ticker, self.permno)
        except:
            display(md("–––––––––Failed to Calculate Beta–––––––––"))
            print(sys.exc_info()[1])
        return
    
    @staticmethod 
    def __get_permno(self, ticker)->float:
        sql_string = """
                    SELECT permno, conm
                    FROM comp.company c
                    INNER JOIN (
                            SELECT DISTINCT permno, gvkey
                            FROM wrdsapps_finratio.firm_ratio
                            WHERE ticker = '{tic}'
                        ) x ON x.gvkey = c.gvkey
                    """.format(tic = ticker)
        data =  connection.raw_sql(sql_string)
        return data['permno'][0]

    @staticmethod
    def __get_company_info(self)->None:
        sql_string = """
                        SELECT DISTINCT 
                            company.conm, ratio.*, company.sic, company.spcsrc, 
                            company.ggroup, company.gind, company.gsector, company.gsubind
                        FROM comp.company AS company
                        INNER JOIN (
                            SELECT permno, gvkey, mktcap, public_date, efftax, de_ratio, roe, int_debt as rod, roa 
                            FROM wrdsapps_finratio.firm_ratio
                            WHERE ticker = '{tic}'
                                AND public_date = (
                                    SELECT max(public_date)
                                    FROM wrdsapps_finratio.firm_ratio
                                )
                        ) ratio ON ratio.gvkey = company.gvkey
                        FETCH FIRST 10 ROWS ONLY""".format(tic = self.ticker)
        identifier = connection.raw_sql(sql_string)
        
        self.name, self.permno, self.gvkey, self.mrktcap, self.public_date,\
            self.efftax, self.de_ratio, self.roe, self.rod, self.roa,\
            self.sic, self.spcsrc, self.ggroup, self.gind, self.gsector, self.gsubind = identifier.values[0]

        self.efftax = float(self.efftax)
        self.public_date = pd.to_datetime(self.public_date)
        self.mrktcap = float(self.mrktcap)
        self.de_ratio = float(self.de_ratio)
        self.efftax
        self.roe
        self.rod
        self.roa

        self.sic = int(self.sic)
        self.gsector = int(self.gsector)
        self.ggroup = int(self.ggroup)
        self.gind = int(self.gind)
        self.gsubind = int(self.gsubind)
        return identifier

    @staticmethod
    def __get_comps_info(self)->None:
        sql_string = """
                        SELECT 
                            date.de_difference, date.permno, date.gvkey, date.ticker, company.conm, date.public_date, 
                            date.mktcap, date.efftax, date.de_ratio, date.roe, date.rod, date.roa,
                            company.sic, company.spcsrc, company.gsector, company.ggroup, company.gind, company.gsubind
                        FROM (
                            SELECT DISTINCT conm, gvkey,
                            sic, spcsrc, ggroup, gind, gsector, gsubind,
                            ABS(CAST(gsector AS INT) - {gsector}) gsector_diff,
                            ABS(CAST(ggroup AS INT) - {ggroup}) ggroup_diff
                            FROM comp.company
                            WHERE sic = {sics}
                                AND ggroup  = {ggroup}
                                AND gvkey <> {gvkey}
                                AND spcsrc IS NOT NULL
                        ) company
                        INNER JOIN ( 
                            SELECT 
                                r.permno, r.ticker, r.gvkey, r.public_date, r.mktcap,
                                ABS(CAST(r.mktcap AS INT) - {mktcap}) as market_cap_difference,
                                ABS(CAST(r.de_ratio AS INT) - {de_ratio}) as de_difference,
                                r.efftax, r.de_ratio, r.roa, r.roe, r.int_debt as rod
                            FROM wrdsapps_finratio.firm_ratio r
                            INNER JOIN (
                                SELECT gvkey, max(public_date) as public_date
                                FROM wrdsapps_finratio.firm_ratio
                                GROUP BY gvkey
                            ) x ON x.gvkey = r.gvkey AND x.public_date = r.public_date
                            ORDER BY market_cap_difference ASC
                        ) date ON date.gvkey = company.gvkey
                        WHERE company.gvkey <> {gvkey}
                        ORDER BY date.market_cap_difference ASC, date.de_difference ASC, spcsrc ASC,
                            gsector DESC, ggroup DESC, gsector_diff DESC, ggroup_diff DESC, date.public_date DESC
                        FETCH FIRST 7 ROWS ONLY""".format(sics = "'"+str(self.sic)+"'",
                                                            gsector = "'"+str(self.gsector)+"'",
                                                            ggroup =  "'"+str(self.ggroup)+"'",
                                                            gind =  "'"+str(self.gind)+"'",
                                                            gsubind =  self.gsubind,
                                                            gvkey = "'"+str(self.gvkey)+"'",
                                                            mktcap = self.mrktcap,
                                                            de_ratio = self.de_ratio
                                                        )
        self.comps = connection.raw_sql(sql_string)
        return

    @staticmethod
    def __get_data(self, ticker, permno)->pd.DataFrame:
        sql_string = """
                        SELECT 
                            date, ret AS {tic}
                        FROM crsp.dsf 
                        WHERE permno = {permno}
                            AND date >= '{start_date}'
                            AND date <= '{end_date}'""".format(
                                                            tic = ticker.upper(),
                                                            permno = permno,
                                                            start_date = self._start_date,
                                                            end_date = self._end_date
                                                        )
        ret_data = connection.raw_sql(sql_string).set_index('date').sort_index(ascending = True)
        return ret_data

    @staticmethod
    def __getbeta(self, ticker, permno, show_plot = False)->float:
        daily_ret = pd.DataFrame(columns=[ticker, self._market, self._rf])
        daily_ret[ticker] = self.__get_data(self, ticker, permno)
        daily_ret[self._market] = self.__get_data(self, self._market,self._market_permno)
        daily_ret[self._rf] = self.__get_data(self, self._rf,self._rf_permno)
        daily_ret.fillna(0.0, inplace = True)
        
        ## Calculate risk premimum
        daily_ret = daily_ret.apply(lambda row: (row[:-1]-(row[self._rf])), axis = 1)
        # data = data.apply(lambda row: (row[:-1]-(row[tbonds]/100)), axis = 1) # Converting bips to %age
        self.debug = daily_ret
        ## Regress OLS, implement CAPM
        formula = ticker + ' ~ '+ self._market
        md("\t"+formula)
        y, X = dmatrices(formula, data=daily_ret, return_type= "dataframe")
        model = sm.OLS(y, X)
        result = model.fit()
        # print(result.summary(),end="\n––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––\n\n")
        
        if show_plot:
            self.__plot_graph(self, ticker, X, y, result.params)
        return result.params[self._market]

    @staticmethod
    def __plot_graph(self, ticker, X, y, params)->None:
        '''
            Function to print the plots
        '''
        try:
            plt.figure(figsize=(10,7))
            plt.scatter(X[self._market], y, label = ticker)
            plt.legend()
            plt.xlabel("Systemic Risk")
            plt.ylabel("Risk Premium")
            plt.title("Security Characteristics Line Regression: "+ticker+" (β: "+str(round(params[self._market], 2))+")")
            plt.axhline(y=0, color='k', linewidth = 0.7)
            plt.axvline(x=0, color='k', linewidth = 0.7)

            f_x = np.dot(X,np.asarray(params))
            plt.plot(X[self._market],f_x, color = "red", linestyle = "dashed", linewidth = 0.7)
        except:
            print(sys.exc_info())
            raise Exception
        return

    @staticmethod
    def __unlever_beta(de: float, tax: float, beta_eq: float, beta_dbt:float = 0.3, method: str = "const de")-> float:
        ## ASK USER FOR TYPE OF UNLEVER TO USE
        if method == "const de":
            beta_unlevered =(beta_eq/(1+de)) + ((beta_dbt*de)/(1+de))
        return beta_unlevered
    
    @staticmethod
    def __relever_beta(de: float, tax: float, beta_unlevered: float, beta_dbt:float = 0.3, method: str = "const de")-> float:
        if method == "const de":
            beta_relevered = beta_unlevered*(1+de) - beta_dbt*de
        return beta_relevered

    @classmethod
    def get_comps_beta(self, verbose = False):

        display(md(r'**Step 1:** Calculating $\beta^{Comps}_{Levered}$'))
        self.comps["beta_levered"] = self.comps[["ticker", "permno"]]\
                                        .apply(lambda row: (self.__getbeta(self,
                                                                            row["ticker"],
                                                                            row["permno"],
                                                                            show_plot = verbose
                                                                        )
                                                            ), axis = 1)

        
        display(md(r'**Step 2:** Unlever $\beta^{Comps}_{Levered}$; Calculating $\beta^{Comps}_{Unlevered}$'))
        
        self.comps["beta_unlevered"] = self.comps[["de_ratio","beta_levered","efftax"]]\
                                            .apply(lambda row: (self.__unlever_beta(de = row["de_ratio"],
                                                                                beta_eq = row["beta_levered"],
                                                                                tax = row["efftax"]
                                                                            )
                                                                ), axis = 1)
        beta_unlevered = self.comps["beta_unlevered"].mean()
        
        display(md(r'**Step 3:** $\beta^{Comps}_{Unlevered} = \beta^{Sector}_{Unlevered} = %f$'%(beta_unlevered)))
        
        beta_relevered_from_comps = self.__relever_beta(de = self.de_ratio,
                                                beta_unlevered = beta_unlevered,
                                                tax = self.efftax)
        display(md(r'**Step 4:** Relever $\beta^{Sector}_{Unlevered} = \beta^{%s}_{Unlevered} \rightarrow \beta^{%s}_{Relevered} = %f | %f$'%(self.ticker, self.ticker, beta_relevered_from_comps, self.beta)))
        return beta_relevered_from_comps

    @classmethod
    def get_roe(self, beta):
        daily_ret = pd.DataFrame(columns=[self._market, self._rf])
        daily_ret[self._market] = self.__get_data(self, self._market,self._market_permno)
        daily_ret[self._rf] = self.__get_data(self, self._rf,self._rf_permno)
        daily_ret.fillna(0.0, inplace = True)
        market_risk_premium = daily_ret[self._market].mean()

        rf = 0
        cost_of_equity = rf + (beta * market_risk_premium)

        display(md(r'Calculated roe: %f'%(cost_of_equity)))
        return cost_of_equity

    @classmethod
    def echo(self)-> None:
        print("\nMarket Ticker:", self._market)
        print("Market Permno:", self._market_permno)
        
        print("\nRisk Ticker:", self._rf)
        print("Risk Free Rate Permno:", self._rf_permno)

        print("\nCompany Name:", self.name)
        print("Ticker:", self.ticker)
        print("Company PermNo:", self.permno)
        print("Company GV-Key: ", self.gvkey)
        print("Market Cap: ", self.mrktcap)
        print("Public Date: ", self.public_date.strftime("%d-%m-%Y"))
        print("Credit Rating: ", self.spcsrc)

        print("\nSIC Code: ", self.sic)
        print("Sector Code: ", self.gsector)
        print("Group Code: ", self.ggroup)
        print("Industry Code: ", self.gind)
        print("Sector Sub Industry Code: ", self.gsubind)
        
        print("\nD/E Ratio:", self.de_ratio)
        print("Effective Tax Rate: ", self.efftax)
        print("Cost of Equity: ", self.roe)
        print("Cost of Debt: ", self.rod)
        print("Cost of Capital: ", self.roa)
        print("Beta: ", self.beta)
        print("Comps:")
        return self.comps

# Initialising Securities

In [34]:
# security = [Security(stk, start_date = start, end_date = end) for stk in stocks][0]
security = Security(stocks[0], start_date = start, end_date = end)
security.echo()
security.comps

......Fetching Company Info......

......Fetching Comps Info......

......Calculating Beta......


Market Ticker: SPY
Market Permno: 84398

Risk Ticker: IRX
Risk Free Rate Permno: 1

Company Name: MICROSOFT CORP
Ticker: MSFT
Company PermNo: 10107.0
Company GV-Key:  012141
Market Cap:  1681605.5064763185
Public Date:  31-12-2020
Credit Rating:  A-

SIC Code:  7372
Sector Code:  45
Group Code:  4510
Industry Code:  451030
Sector Sub Industry Code:  45103020

D/E Ratio: 1.5102722246765
Effective Tax Rate:  0.165076551776152
Cost of Equity:  0.4223556089102308
Cost of Debt:  0.03306311635798657
Cost of Capital:  0.2389846313785961
Beta:  1.2221886102450663
Comps:


,de_difference,permno,gvkey,ticker,conm,public_date,mktcap,efftax,de_ratio,roe,rod,roa,sic,spcsrc,gsector,ggroup,gind,gsubind
0,0.510272,90215.0,157855,CRM,SALESFORCE.COM INC,2020-12-31,203614.948883,0.821530,0.533050,0.101506,0.022609,0.052404,7372,B-,45,4510,451030,45103010
1,0.510272,78975.0,027928,INTU,INTUIT INC,2020-12-31,99804.069704,0.169245,0.865395,0.453409,0.014722,0.332626,7372,A-,45,4510,451030,45103010
2,0.510272,77357.0,024975,SNPS,SYNOPSYS INC,2020-12-31,39564.688830,-0.039626,0.635304,0.147726,0.015044,0.120581,7372,B,45,4510,451030,45103010
3,0.510272,11403.0,013421,CDNS,CADENCE DESIGN SYSTEMS INC,2020-12-31,38054.554287,-1.064812,0.712604,0.538360,0.054689,0.213861,7372,B,45,4510,451030,45103010
4,0.489728,87184.0,122841,RHT,RED HAT INC,2019-07-31,33427.959930,0.162274,2.462787,0.320664,0.049283,0.134344,7372,B+,45,4510,451030,45103020
5,1.510272,83621.0,063080,ANSS,ANSYS INC,2020-12-31,31244.598152,0.136415,0.370987,0.115059,0.021809,0.115101,7372,B+,45,4510,451030,45103010
6,0.489728,93312.0,062965,SSNC,SS&C TECHNOLOGIES HLDGS INC,2020-12-31,18621.453750,0.175287,2.050768,0.092822,0.040425,0.108798,7372,B+,45,4510,451030,45103010


# Compute Cost of Equity

In [35]:
beta_relevered_from_comps = security.get_comps_beta()

**Step 1:** Calculating $\beta^{Comps}_{Levered}$

**Step 2:** Unlever $\beta^{Comps}_{Levered}$; Calculating $\beta^{Comps}_{Unlevered}$

**Step 3:** $\beta^{Comps}_{Unlevered} = \beta^{Sector}_{Unlevered} = 0.738039$

**Step 4:** Relever $\beta^{Sector}_{Unlevered} = \beta^{MSFT}_{Unlevered} \rightarrow \beta^{MSFT}_{Relevered} = 1.399596 | 1.222189$

In [36]:
coe_comps = security.get_roe(beta_relevered_from_comps)
coe_scl = security.get_roe(security.beta)

Calculated roe: 0.000757

Calculated roe: 0.000661

In [37]:
print(coe_comps*252, coe_scl*252)

0.19080357281382773 0.1666180432159064


***
***

In [ ]:
def get_data(ticker, permno, period)->pd.DataFrame:
    sql_string = """
                    SELECT 
                        date, ret AS {tic}
                    FROM crsp.dsf 
                    WHERE permno = {permno}
                        AND date >= '2015-01-01'
                        AND date <= '2021-01-01'""".format(
                                                        tic = ticker.upper(),
                                                        permno = permno
                                                    )
    ret_data = connection.raw_sql(sql_string).set_index('date').sort_index(ascending = True)
    ret_data.resample('3T')
    return ret_data

get_data('SPY')